In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from tqdm.auto import tqdm

from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [3]:
df = pd.read_csv("/Users/mikhailmarin/Downloads/train.csv")
df.shape

(60487, 4)

In [4]:
def mirror_ts(data, n_points=25, x_col='x'):
    mirrored_df = []
    border = [0 for _ in range(n_points)]
    for _id, group in data.groupby('id'):
        x_list = group[x_col].to_list()
        x = x_list[:n_points][::-1]
        x += x_list
        x += x_list[::-1][:n_points]
        tmp = pd.DataFrame(x, columns=[x_col])
        tmp['id'] = _id
        tmp['y'] = border + group['y'].to_list() + border
        tmp['original'] = border + [1 for _ in range(group.shape[0])] + border
        mirrored_df.append(tmp)
    mirrored_df = pd.concat(mirrored_df).reset_index(drop=True)[['id', x_col, 'y', 'original']]
    return mirrored_df

In [5]:
def ts_features(dataframe, param, x_col='x', mirror_param=25):
    
    data = mirror_ts(dataframe, n_points=mirror_param, x_col=x_col)
    
    groups = data.groupby('id')
    for i in range(-param, param + 1):
        if i == 0:
            continue
        data[f'{x_col}_shift_{i}'] = groups[x_col].shift(i)
        data[f'{x_col}_delta_{i}'] = groups[x_col].diff(i)
        
    for i in range(-3, 4):
        for j in range(-3, 4):
            if i == 0 or j == 0 or i == j:
                continue
            values = (data[f'{x_col}_delta_{i}'] / (data[f'{x_col}_delta_{j}'] + 1e-9)).values
            data[f'{x_col}_delta_{i}_divide_delta_{j}'] = values
            
            values = (data[f'{x_col}_delta_{i}'] - data[f'{x_col}_delta_{j}']).values
            data[f'{x_col}_delta_{i}_substract_delta_{j}'] = values
            
            values = (data[f'{x_col}_shift_{i}'] / (data[f'{x_col}_shift_{j}'] + 1e-9)).values
            data[f'{x_col}_delta_{i}_divide_shift_{j}'] = values
            
            values = (data[f'{x_col}_shift_{i}'] - data[f'{x_col}_shift_{j}']).values
            data[f'{x_col}_shift_{i}_substract_shift_{j}'] = values
        
    data[f'{x_col}_dist_to_lower'] = data[x_col] - 400
    data[f'{x_col}_dist_to_upper'] = 1200 - data[x_col]
    
    for i in range(-3, 4):
        if i == 0:
            continue
        data[f'{x_col}_substract_shift_{i}'] = data[x_col] - data[f'{x_col}_shift_{i}']
        data[f'{x_col}_divide_shift_{i}'] = data[x_col] / (data[f'{x_col}_shift_{i}'] + 1e-9)
        data[f'{x_col}_substract_delta_{i}'] = data[x_col] - data[f'{x_col}_delta_{i}']
        data[f'{x_col}_divide_delta_{i}'] = data[x_col] / (data[f'{x_col}_delta_{i}'] + 1e-9)
    
    drop_columns = ['id', x_col, 'y', 'original']
    return data.loc[data['original'] == 1].drop(columns=drop_columns).reset_index(drop=True)


def window_features(dataframe, param=10, x_col='x', mirror_param=25):
    
    data = mirror_ts(dataframe, n_points=mirror_param, x_col=x_col)
    created_columns = []
    groups = data.groupby('id')[x_col]
    for i in range(2, param + 1):
        data[f'{x_col}_mean_{i}'] = groups.rolling(window=i).mean().values
        data[f'{x_col}_median_{i}'] = groups.rolling(window=i).median().values
        data[f'{x_col}_std_{i}'] = groups.rolling(window=i).std().values
        data[f'{x_col}_min_{i}'] = groups.rolling(window=i).min().values
        data[f'{x_col}_max_{i}'] = groups.rolling(window=i).max().values
        data[f'{x_col}_r_{i}'] = data[f'{x_col}_max_{i}'] - data[f'{x_col}_min_{i}']
        data[f'{x_col}_z_score_v1'] = (data[x_col] - data[f'{x_col}_mean_{i}']) / data[f'{x_col}_std_{i}']
        data[f'{x_col}_z_score_v2'] = (data[x_col] - data[f'{x_col}_median_{i}']) / data[f'{x_col}_std_{i}']
        data[f'{x_col}_z_score_v3'] = (data[x_col] - data[f'{x_col}_mean_{i}']) / data[f'{x_col}_r_{i}']
        data[f'{x_col}_z_score_v4'] = (data[x_col] - data[f'{x_col}_median_{i}']) / data[f'{x_col}_r_{i}']
        
        created_columns.extend(
            [
                f'{x_col}_mean_{i}', f'{x_col}_median_{i}', 
                f'{x_col}_std_{i}', f'{x_col}_min_{i}', 
                f'{x_col}_max_{i}', f'{x_col}_r_{i}',
                f'{x_col}_z_score_v1', f'{x_col}_z_score_v2', 
                f'{x_col}_z_score_v3', f'{x_col}_z_score_v4'
            ]
        )
    for col in created_columns:
        groups = data.groupby('id')[col]
        for i in [-2, -1, 1, 2]:
            data[f'{col}_shift_{i}'] = groups.shift(i).values
    drop_columns = ['id', x_col, 'y', 'original']
    return data.loc[data['original'] == 1].drop(columns=drop_columns).reset_index(drop=True)

In [6]:
def smooth_ts(dataframe, mirror_param=25):
    data = mirror_ts(dataframe)
    df_smooth = []
    for _id, group in data.groupby('id'):
        smoothed = group['x'].rolling(21).median().fillna(method='bfill').shift(-10).fillna(method='ffill')
        mask = group['x'] / smoothed
        group['mask'] = mask.values
        group['smooth'] = smoothed.values
        group.loc[group['mask'] > 1.75, 'x'] = group.loc[group['mask'] > 1.75, 'smooth']
        group = group.drop(columns = ['mask','smooth'])
        df_smooth.append(group)
    df_smooth = pd.concat(df_smooth, ignore_index=True).reset_index(drop=True).rename(columns={'x': 'x_smooth'})
    return df_smooth.loc[df_smooth['original'] == 1].drop(columns='original').reset_index(drop=True)

In [7]:
def log_ts(dataframe):
    df_log = dataframe.copy(deep=True)
    df_log['x_log'] = np.log10(df_log['x'].values)
    df_log = df_log.drop(columns=['x'])
    return df_log

In [8]:
df_log = log_ts(df)
df_smooth = smooth_ts(df)

In [9]:
ts_features_df = ts_features(dataframe=df, param=10, x_col='x', mirror_param=25)
window_features_df = window_features(dataframe=df, param=10, x_col='x', mirror_param=25)
df = pd.concat([df, ts_features_df, window_features_df], axis=1)
df.head()

,id,time,x,y,x_shift_-10,x_delta_-10,x_shift_-9,x_delta_-9,x_shift_-8,x_delta_-8,x_shift_-7,x_delta_-7,x_shift_-6,x_delta_-6,x_shift_-5,x_delta_-5,x_shift_-4,x_delta_-4,x_shift_-3,x_delta_-3,x_shift_-2,x_delta_-2,x_shift_-1,x_delta_-1,x_shift_1,x_delta_1,x_shift_2,x_delta_2,x_shift_3,x_delta_3,x_shift_4,x_delta_4,x_shift_5,x_delta_5,x_shift_6,x_delta_6,x_shift_7,x_delta_7,x_shift_8,x_delta_8,x_shift_9,x_delta_9,x_shift_10,x_delta_10,x_delta_-3_divide_delta_-2,x_delta_-3_substract_delta_-2,x_delta_-3_divide_shift_-2,x_shift_-3_substract_shift_-2,x_delta_-3_divide_delta_-1,x_delta_-3_substract_delta_-1,x_delta_-3_divide_shift_-1,x_shift_-3_substract_shift_-1,x_delta_-3_divide_delta_1,x_delta_-3_substract_delta_1,x_delta_-3_divide_shift_1,x_shift_-3_substract_shift_1,x_delta_-3_divide_delta_2,x_delta_-3_substract_delta_2,x_delta_-3_divide_shift_2,x_shift_-3_substract_shift_2,x_delta_-3_divide_delta_3,x_delta_-3_substract_delta_3,x_delta_-3_divide_shift_3,x_shift_-3_substract_shift_3,x_delta_-2_divide_delta_-3,x_delta_-2_substract_delta_-3,x_delta_-2_divide_shift_-3,x_shift_-2_substract_shift_-3,x_delta_-2_divide_delta_-1,x_delta_-2_substract_delta_-1,x_delta_-2_divide_shift_-1,x_shift_-2_substract_shift_-1,x_delta_-2_divide_delta_1,x_delta_-2_substract_delta_1,x_delta_-2_divide_shift_1,x_shift_-2_substract_shift_1,x_delta_-2_divide_delta_2,x_delta_-2_substract_delta_2,x_delta_-2_divide_shift_2,x_shift_-2_substract_shift_2,x_delta_-2_divide_delta_3,x_delta_-2_substract_delta_3,x_delta_-2_divide_shift_3,x_shift_-2_substract_shift_3,x_delta_-1_divide_delta_-3,x_delta_-1_substract_delta_-3,x_delta_-1_divide_shift_-3,x_shift_-1_substract_shift_-3,x_delta_-1_divide_delta_-2,x_delta_-1_substract_delta_-2,x_delta_-1_divide_shift_-2,x_shift_-1_substract_shift_-2,x_delta_-1_divide_delta_1,x_delta_-1_substract_delta_1,x_delta_-1_divide_shift_1,x_shift_-1_substract_shift_1,x_delta_-1_divide_delta_2,x_delta_-1_substract_delta_2,x_delta_-1_divide_shift_2,x_shift_-1_substract_shift_2,x_delta_-1_divide_delta_3,x_delta_-1_substract_delta_3,x_delta_-1_divide_shift_3,x_shift_-1_substract_shift_3,x_delta_1_divide_delta_-3,x_delta_1_substract_delta_-3,x_delta_1_divide_shift_-3,x_shift_1_substract_shift_-3,x_delta_1_divide_delta_-2,x_delta_1_substract_delta_-2,x_delta_1_divide_shift_-2,x_shift_1_substract_shift_-2,x_delta_1_divide_delta_-1,x_delta_1_substract_delta_-1,x_delta_1_divide_shift_-1,x_shift_1_substract_shift_-1,x_delta_1_divide_delta_2,x_delta_1_substract_delta_2,x_delta_1_divide_shift_2,x_shift_1_substract_shift_2,x_delta_1_divide_delta_3,x_delta_1_substract_delta_3,x_delta_1_divide_shift_3,x_shift_1_substract_shift_3,x_delta_2_divide_delta_-3,x_delta_2_substract_delta_-3,x_delta_2_divide_shift_-3,x_shift_2_substract_shift_-3,x_delta_2_divide_delta_-2,x_delta_2_substract_delta_-2,x_delta_2_divide_shift_-2,x_shift_2_substract_shift_-2,x_delta_2_divide_delta_-1,x_delta_2_substract_delta_-1,x_delta_2_divide_shift_-1,x_shift_2_substract_shift_-1,x_delta_2_divide_delta_1,x_delta_2_substract_delta_1,x_delta_2_divide_shift_1,x_shift_2_substract_shift_1,x_delta_2_divide_delta_3,x_delta_2_substract_delta_3,x_delta_2_divide_shift_3,x_shift_2_substract_shift_3,x_delta_3_divide_delta_-3,x_delta_3_substract_delta_-3,x_delta_3_divide_shift_-3,x_shift_3_substract_shift_-3,x_delta_3_divide_delta_-2,x_delta_3_substract_delta_-2,x_delta_3_divide_shift_-2,x_shift_3_substract_shift_-2,x_delta_3_divide_delta_-1,x_delta_3_substract_delta_-1,x_delta_3_divide_shift_-1,x_shift_3_substract_shift_-1,x_delta_3_divide_delta_1,x_delta_3_substract_delta_1,x_delta_3_divide_shift_1,x_shift_3_substract_shift_1,x_delta_3_divide_delta_2,x_delta_3_substract_delta_2,x_delta_3_divide_shift_2,x_shift_3_substract_shift_2,x_dist_to_lower,x_dist_to_upper,x_substract_shift_-3,x_divide_shift_-3,x_substract_delta_-3,x_divide_delta_-3,x_substract_shift_-2,x_divide_shift_-2,x_substract_delta_-2,x_divide_delta_-2,x_substract_shift_-1,x_divide_shift_-1,x_substract_delta_-1,x_divide_delta_

In [10]:
ts_features_df = ts_features(dataframe=df_log, param=10, x_col='x_log', mirror_param=25)
window_features_df = window_features(dataframe=df_log, param=10, x_col='x_log', mirror_param=25)
df_log = pd.concat([df_log, ts_features_df, window_features_df], axis=1)
df_log.head()

,id,time,y,x_log,x_log_shift_-10,x_log_delta_-10,x_log_shift_-9,x_log_delta_-9,x_log_shift_-8,x_log_delta_-8,x_log_shift_-7,x_log_delta_-7,x_log_shift_-6,x_log_delta_-6,x_log_shift_-5,x_log_delta_-5,x_log_shift_-4,x_log_delta_-4,x_log_shift_-3,x_log_delta_-3,x_log_shift_-2,x_log_delta_-2,x_log_shift_-1,x_log_delta_-1,x_log_shift_1,x_log_delta_1,x_log_shift_2,x_log_delta_2,x_log_shift_3,x_log_delta_3,x_log_shift_4,x_log_delta_4,x_log_shift_5,x_log_delta_5,x_log_shift_6,x_log_delta_6,x_log_shift_7,x_log_delta_7,x_log_shift_8,x_log_delta_8,x_log_shift_9,x_log_delta_9,x_log_shift_10,x_log_delta_10,x_log_delta_-3_divide_delta_-2,x_log_delta_-3_substract_delta_-2,x_log_delta_-3_divide_shift_-2,x_log_shift_-3_substract_shift_-2,x_log_delta_-3_divide_delta_-1,x_log_delta_-3_substract_delta_-1,x_log_delta_-3_divide_shift_-1,x_log_shift_-3_substract_shift_-1,x_log_delta_-3_divide_delta_1,x_log_delta_-3_substract_delta_1,x_log_delta_-3_divide_shift_1,x_log_shift_-3_substract_shift_1,x_log_delta_-3_divide_delta_2,x_log_delta_-3_substract_delta_2,x_log_delta_-3_divide_shift_2,x_log_shift_-3_substract_shift_2,x_log_delta_-3_divide_delta_3,x_log_delta_-3_substract_delta_3,x_log_delta_-3_divide_shift_3,x_log_shift_-3_substract_shift_3,x_log_delta_-2_divide_delta_-3,x_log_delta_-2_substract_delta_-3,x_log_delta_-2_divide_shift_-3,x_log_shift_-2_substract_shift_-3,x_log_delta_-2_divide_delta_-1,x_log_delta_-2_substract_delta_-1,x_log_delta_-2_divide_shift_-1,x_log_shift_-2_substract_shift_-1,x_log_delta_-2_divide_delta_1,x_log_delta_-2_substract_delta_1,x_log_delta_-2_divide_shift_1,x_log_shift_-2_substract_shift_1,x_log_delta_-2_divide_delta_2,x_log_delta_-2_substract_delta_2,x_log_delta_-2_divide_shift_2,x_log_shift_-2_substract_shift_2,x_log_delta_-2_divide_delta_3,x_log_delta_-2_substract_delta_3,x_log_delta_-2_divide_shift_3,x_log_shift_-2_substract_shift_3,x_log_delta_-1_divide_delta_-3,x_log_delta_-1_substract_delta_-3,x_log_delta_-1_divide_shift_-3,x_log_shift_-1_substract_shift_-3,x_log_delta_-1_divide_delta_-2,x_log_delta_-1_substract_delta_-2,x_log_delta_-1_divide_shift_-2,x_log_shift_-1_substract_shift_-2,x_log_delta_-1_divide_delta_1,x_log_delta_-1_substract_delta_1,x_log_delta_-1_divide_shift_1,x_log_shift_-1_substract_shift_1,x_log_delta_-1_divide_delta_2,x_log_delta_-1_substract_delta_2,x_log_delta_-1_divide_shift_2,x_log_shift_-1_substract_shift_2,x_log_delta_-1_divide_delta_3,x_log_delta_-1_substract_delta_3,x_log_delta_-1_divide_shift_3,x_log_shift_-1_substract_shift_3,x_log_delta_1_divide_delta_-3,x_log_delta_1_substract_delta_-3,x_log_delta_1_divide_shift_-3,x_log_shift_1_substract_shift_-3,x_log_delta_1_divide_delta_-2,x_log_delta_1_substract_delta_-2,x_log_delta_1_divide_shift_-2,x_log_shift_1_substract_shift_-2,x_log_delta_1_divide_delta_-1,x_log_delta_1_substract_delta_-1,x_log_delta_1_divide_shift_-1,x_log_shift_1_substract_shift_-1,x_log_delta_1_divide_delta_2,x_log_delta_1_substract_delta_2,x_log_delta_1_divide_shift_2,x_log_shift_1_substract_shift_2,x_log_delta_1_divide_delta_3,x_log_delta_1_substract_delta_3,x_log_delta_1_divide_shift_3,x_log_shift_1_substract_shift_3,x_log_delta_2_divide_delta_-3,x_log_delta_2_substract_delta_-3,x_log_delta_2_divide_shift_-3,x_log_shift_2_substract_shift_-3,x_log_delta_2_divide_delta_-2,x_log_delta_2_substract_delta_-2,x_log_delta_2_divide_shift_-2,x_log_shift_2_substract_shift_-2,x_log_delta_2_divide_delta_-1,x_log_delta_2_substract_delta_-1,x_log_delta_2_divide_shift_-1,x_log_shift_2_substract_shift_-1,x_log_delta_2_divide_delta_1,x_log_delta_2_substract_delta_1,x_log_delta_2_divide_shift_1,x_log_shift_2_substract_shift_1,x_log_delta_2_divide_delta_3,x_log_delta_2_substract_delta_3,x_log_delta_2_divide_shift_3,x_log_shift_2_substract_shift_3,x_log_delta_3_divide_delta_-3,x_log_delta_3_substract_delta_-3,x_log_delta_3_divide_shift_-3,x_log_shift_3_substract_shift_-3,x_log_delta_3_divide_delta_-2,x_log_delta_3_substract_delta_-2,x_log_delta_3_divide_shift_-2,x_log_shift_3_substrac

In [11]:
ts_features_df = ts_features(dataframe=df_smooth, param=10, x_col='x_smooth', mirror_param=25)
window_features_df = window_features(dataframe=df_smooth, param=10, x_col='x_smooth', mirror_param=25)
df_smooth = pd.concat([df_smooth, ts_features_df, window_features_df], axis=1)
df_smooth.head()

,id,x_smooth,y,x_smooth_shift_-10,x_smooth_delta_-10,x_smooth_shift_-9,x_smooth_delta_-9,x_smooth_shift_-8,x_smooth_delta_-8,x_smooth_shift_-7,x_smooth_delta_-7,x_smooth_shift_-6,x_smooth_delta_-6,x_smooth_shift_-5,x_smooth_delta_-5,x_smooth_shift_-4,x_smooth_delta_-4,x_smooth_shift_-3,x_smooth_delta_-3,x_smooth_shift_-2,x_smooth_delta_-2,x_smooth_shift_-1,x_smooth_delta_-1,x_smooth_shift_1,x_smooth_delta_1,x_smooth_shift_2,x_smooth_delta_2,x_smooth_shift_3,x_smooth_delta_3,x_smooth_shift_4,x_smooth_delta_4,x_smooth_shift_5,x_smooth_delta_5,x_smooth_shift_6,x_smooth_delta_6,x_smooth_shift_7,x_smooth_delta_7,x_smooth_shift_8,x_smooth_delta_8,x_smooth_shift_9,x_smooth_delta_9,x_smooth_shift_10,x_smooth_delta_10,x_smooth_delta_-3_divide_delta_-2,x_smooth_delta_-3_substract_delta_-2,x_smooth_delta_-3_divide_shift_-2,x_smooth_shift_-3_substract_shift_-2,x_smooth_delta_-3_divide_delta_-1,x_smooth_delta_-3_substract_delta_-1,x_smooth_delta_-3_divide_shift_-1,x_smooth_shift_-3_substract_shift_-1,x_smooth_delta_-3_divide_delta_1,x_smooth_delta_-3_substract_delta_1,x_smooth_delta_-3_divide_shift_1,x_smooth_shift_-3_substract_shift_1,x_smooth_delta_-3_divide_delta_2,x_smooth_delta_-3_substract_delta_2,x_smooth_delta_-3_divide_shift_2,x_smooth_shift_-3_substract_shift_2,x_smooth_delta_-3_divide_delta_3,x_smooth_delta_-3_substract_delta_3,x_smooth_delta_-3_divide_shift_3,x_smooth_shift_-3_substract_shift_3,x_smooth_delta_-2_divide_delta_-3,x_smooth_delta_-2_substract_delta_-3,x_smooth_delta_-2_divide_shift_-3,x_smooth_shift_-2_substract_shift_-3,x_smooth_delta_-2_divide_delta_-1,x_smooth_delta_-2_substract_delta_-1,x_smooth_delta_-2_divide_shift_-1,x_smooth_shift_-2_substract_shift_-1,x_smooth_delta_-2_divide_delta_1,x_smooth_delta_-2_substract_delta_1,x_smooth_delta_-2_divide_shift_1,x_smooth_shift_-2_substract_shift_1,x_smooth_delta_-2_divide_delta_2,x_smooth_delta_-2_substract_delta_2,x_smooth_delta_-2_divide_shift_2,x_smooth_shift_-2_substract_shift_2,x_smooth_delta_-2_divide_delta_3,x_smooth_delta_-2_substract_delta_3,x_smooth_delta_-2_divide_shift_3,x_smooth_shift_-2_substract_shift_3,x_smooth_delta_-1_divide_delta_-3,x_smooth_delta_-1_substract_delta_-3,x_smooth_delta_-1_divide_shift_-3,x_smooth_shift_-1_substract_shift_-3,x_smooth_delta_-1_divide_delta_-2,x_smooth_delta_-1_substract_delta_-2,x_smooth_delta_-1_divide_shift_-2,x_smooth_shift_-1_substract_shift_-2,x_smooth_delta_-1_divide_delta_1,x_smooth_delta_-1_substract_delta_1,x_smooth_delta_-1_divide_shift_1,x_smooth_shift_-1_substract_shift_1,x_smooth_delta_-1_divide_delta_2,x_smooth_delta_-1_substract_delta_2,x_smooth_delta_-1_divide_shift_2,x_smooth_shift_-1_substract_shift_2,x_smooth_delta_-1_divide_delta_3,x_smooth_delta_-1_substract_delta_3,x_smooth_delta_-1_divide_shift_3,x_smooth_shift_-1_substract_shift_3,x_smooth_delta_1_divide_delta_-3,x_smooth_delta_1_substract_delta_-3,x_smooth_delta_1_divide_shift_-3,x_smooth_shift_1_substract_shift_-3,x_smooth_delta_1_divide_delta_-2,x_smooth_delta_1_substract_delta_-2,x_smooth_delta_1_divide_shift_-2,x_smooth_shift_1_substract_shift_-2,x_smooth_delta_1_divide_delta_-1,x_smooth_delta_1_substract_delta_-1,x_smooth_delta_1_divide_shift_-1,x_smooth_shift_1_substract_shift_-1,x_smooth_delta_1_divide_delta_2,x_smooth_delta_1_substract_delta_2,x_smooth_delta_1_divide_shift_2,x_smooth_shift_1_substract_shift_2,x_smooth_delta_1_divide_delta_3,x_smooth_delta_1_substract_delta_3,x_smooth_delta_1_divide_shift_3,x_smooth_shift_1_substract_shift_3,x_smooth_delta_2_divide_delta_-3,x_smooth_delta_2_substract_delta_-3,x_smooth_delta_2_divide_shift_-3,x_smooth_shift_2_substract_shift_-3,x_smooth_delta_2_divide_delta_-2,x_smooth_delta_2_substract_delta_-2,x_smooth_delta_2_divide_shift_-2,x_smooth_shift_2_substract_shift_-2,x_smooth_delta_2_divide_delta_-1,x_smooth_delta_2_substract_delta_-1,x_smooth_delta_2_divide_shift_-1,x_smooth_shift_2_substract_shift_-1,x_smooth_delta_2_divide_delta_1,x_smooth_delta_2_substract_delta_1,x_smooth_delta_2_divide_

In [12]:
dataset = pd.concat(
    [
        df, 
        df_log.drop(columns=['id', 'time', 'y']),
        df_smooth.drop(columns=['id', 'y']),
    ],
    axis=1
)

In [13]:
del df, df_log, df_smooth, ts_features_df, window_features_df
import gc
gc.collect()

0

In [14]:
columns = dataset.columns

int_columns = ['id', 'time', 'x', 'y']
float_columns = []

float_columns += [c for c in columns if 'mean' in c]
float_columns += [c for c in columns if 'std' in c]
float_columns += [c for c in columns if 'std' in c]
float_columns += [c for c in columns if 'divide' in c]
float_columns += [c for c in columns if 'z_score' in c]
float_columns += [c for c in columns if 'log' in c]

int_columns += [c for c in columns if c not in float_columns]

In [15]:
dtypes = {}
for col in int_columns:
    dtypes[col] =  'int32'
for col in float_columns:
    dtypes[col] = 'float32'

In [16]:
dataset = dataset.astype(dtypes)

In [21]:
dataset['nnfeat'] = pd.read_csv('/Users/mikhailmarin/Downloads/feat.csv')['nnfeat'].values

In [22]:
golden_feature = []
for n, col in enumerate(dataset.drop(columns=['id', 'y', 'time']).columns):
    if col in float_columns:
        golden_feature.append(f'{n}:border_count=512')

In [24]:
from sklearn.metrics import roc_curve, precision_recall_curve, f1_score

def threshold_search(y_true, y_proba):
    precision , recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1 / (precision + 1e-18) + 1 / (recall + 1e-18))
    F[F>1.0] = 0
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    return best_th, best_score

In [25]:
# import optuna

# def objective(trial, data=dataset):
    
#     param = {
#         'learning_rate': trial.suggest_uniform("learning_rate", 1e-3, 1),
#         'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 5),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 500),
#         'max_depth': trial.suggest_int('max_depth', 2, 7)
#     }
#     param['grow_policy'] = 'SymmetricTree'
#     param['objective'] = 'Logloss'
#     param['iterations'] = 3000
#     param['use_best_model'] = True
#     param['per_float_feature_quantization'] = golden_feature
#     param['thread_count'] = -1
#     param['random_seed'] = 13
    
#     scores = []
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)

#     for tr_id, va_id in kf.split(data['id'].unique()):

#         train = data.loc[data['id'].isin(tr_id)].reset_index(drop=True)
#         valid = data.loc[data['id'].isin(va_id)].reset_index(drop=True)

#         x_train, y_train = train.drop(columns=['id', 'y', 'time']).values, train['y'].values
#         x_valid, y_valid = valid.drop(columns=['id', 'y', 'time']).values, valid['y'].values
        
#         model = CatBoostClassifier(**param)
#         model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=50, early_stopping_rounds=100)
#         best_th, _ = threshold_search(y_train, model.predict_proba(x_train)[:, 1])
#         predict = (model.predict_proba(x_valid)[:, 1] > best_th).astype(int)
#         score = f1_score(y_valid, predict)
#         scores.append(score)
#     return np.mean(scores)

In [26]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=30)

In [27]:
best_params = {
    'learning_rate': 0.00522903409129151,
    'l2_leaf_reg': 0.003570650864945256,
    'min_data_in_leaf': 406,
    'max_depth': 7,
    'grow_policy': 'SymmetricTree',
    'objective': 'Logloss',
    'iterations': 5000,
    'use_best_model': True,
    'per_float_feature_quantization': golden_feature,
    'thread_count': -1,
    'random_seed': 13,
}

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []
thresholds = []
feature_importances_global = []

for tr_id, va_id in kf.split(dataset['id'].unique()):

    train = dataset.loc[dataset['id'].isin(tr_id)].reset_index(drop=True)
    valid = dataset.loc[dataset['id'].isin(va_id)].reset_index(drop=True)

    x_train, y_train = train.drop(columns=['id', 'y', 'time']).values, train['y'].values
    x_valid, y_valid = valid.drop(columns=['id', 'y', 'time']).values, valid['y'].values

    catboost_model = CatBoostClassifier(**best_params)
    catboost_model.fit(x_train, y_train, eval_set=(x_valid, y_valid), silent=True, plot=True, early_stopping_rounds=100)
    best_th, _ = threshold_search(y_train, catboost_model.predict_proba(x_train)[:, 1])
    predict = (catboost_model.predict_proba(x_valid)[:, 1] > best_th).astype(int)
    score = f1_score(y_valid, predict)
    scores.append(score)
    thresholds.append(best_th)
    
    feature_importances_global.append(catboost_model.feature_importances_)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [78]:
for f, score in enumerate(scores):
    print(f'Fold {f}. F1 = {score:.4f}')

print(f'Mean = {np.mean(scores):.4f}')
print(f'Std = {np.std(scores):.4f}')

Fold 0. F1 = 0.8081
Fold 1. F1 = 0.6685
Fold 2. F1 = 0.8415
Fold 3. F1 = 0.8648
Fold 4. F1 = 0.8809
Mean = 0.8127
Std = 0.0762


In [ ]:
lgbm_params = {
    'lambda_l1': 1.4100477737746033e-05, 
    'lambda_l2': 0.19031962867065458, 
    'num_leaves': 152, 
    'feature_fraction': 0.9016506322288333, 
    'bagging_fraction': 0.4138061960003289, 
    'bagging_freq': 7, 
    'min_child_samples': 218,
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'n_estimators': 5000
}